# Extract Whistles From Box 2 Box

We need to extract whistles played through the ocean and recorded by a box.

## Basic Idea
Use the original file to find gaps.
Align the recorded file to the original file (manually).
Find all gaps that are large enough in the original file:
    $$(gap_{i, start}, gap_{i, stop})$$.
    
Use the gaps to find whistles. Since the extractor is not perfect, we classify each region using the trace and
a nearest neighbor based solution.

In [186]:
import os
import matplotlib.pyplot as plt
import numpy as np

from mimic_utils.spectrogram import *
from mimic_utils.whistle_tracer import *
from mimic_utils.params import * 
from scipy.io import wavfile
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw


## The algorithms to extract the audio

In [197]:
def wtrace(audio):
    spec = fwd_spectrogram(data_original[start - border: stop + border, 0], FFT_WIN, FFT_STEP)
    whistle_trace, _  = trace(spec, TRACE_RAD, SMOOTH_ENT)   
    lo = min(whistle_trace)
    hi = max(whistle_trace)
    return (whistle_trace - lo) / (hi - lo)


class Classifier:
    
    def __init__(self, folder):
        self.templates = {}
        for filename in os.listdir(folder):
            if filename.endswith(".wav"):
                print("Loading Classifier Template: {}".format(filename))
                basename = filename.split("/")[-1].replace(".wav", "")
                _,  data = wavfile.read("{}/{}".format(folder, filename))
                self.templates[basename] = wtrace(data)
        
    def nn(self, trace):
        min_label = -1
        min_dist  = float('inf') 
        for label, template in self.templates.items():
            dist, _ = fastdtw(trace, template, dist=euclidean)
            if dist < min_dist:
                min_dist = dist
                min_label = label
        return label            

    
def extract_all(original, recorded, output, classifier, border = 30000, min_gap_size = 123500, th = 0.1):
    basename = recorded.split("/")[-1].replace(".wav", "")
    print(basename)
    _,  data_original = wavfile.read(original)
    fs, data_box      = wavfile.read(recorded)
    data = (data_original[:, 0] + 32768) / (32768 + 32767)    

    last_sample = 0
    start_i = 0    
    gaps = []
    
    n = len(data_original)
    for i in range(10, len(data_original)):
        sample = np.sum(data[i - 10:i]) / 10
        if sample < th and last_sample >= th and i - start_i > min_gap_size:
            print("STOP {} {} {} %done {} {}".format(start_i / fs, i / fs, (i - start_i) / fs, (i / n) * 100, len(gaps)))
            gaps.append([start_i, i])        
        if sample >= th and last_sample < th:
            start_i = i
        last_sample = sample
        
    for i in range(1, len(gaps)):  
        _, start = gaps[i - 1]
        stop, _  = gaps[i]
        trace = wtrace(data_original[start - border: stop + border, 0])
        name = classifier.nn(trace)
        filename = '{}/{}_{}_{}.wav'.format(output, basename, start, name)
        print(filename)
        wavfile.write(filename, fs, data_box[start - border: stop + border, 0])

## Extract the 00 variations

In [ ]:
classifier = Classifier("originals/")
output     = "00" 
original   = '/Users/daniel.kohlsdorf/Desktop/00-all-whistles-2019-synth--04--18.wav'
recorded   = '/Users/daniel.kohlsdorf/Desktop/chat1-2019-06-23T123304-192k.wav'
extract_all(original, recorded, output, classifier)

Loading Classifier Template: sar.wav
Loading Classifier Template: sca.wav
Loading Classifier Template: gra.wav
Loading Classifier Template: den.wav
Loading Classifier Template: rop.wav
chat1-2019-06-23T123304-192k
STOP 5.208333333333334e-05 1.9804947916666666 1.9804427083333334 %done 0.5756447507323725 0
STOP 3.669942708333333 4.601833333333333 0.931890625 %done 1.3375552479233388 1
STOP 6.2605 7.185640625 0.925140625 %done 2.0885570231414774 2
STOP 8.826036458333334 9.730442708333333 0.90440625 %done 2.8282216600228827 3
STOP 11.349729166666666 12.282765625 0.9330364583333334 %done 3.5700722800473295 4
STOP 13.919776041666667 14.759625 0.8398489583333333 %done 4.28998889054301 5
STOP 16.01065625 16.765953125 0.755296875 %done 4.873142281434309 6
STOP 17.758463541666668 18.493244791666665 0.73478125 %done 5.3751917617380105 7
STOP 19.474208333333333 20.2138125 0.7396041666666666 %done 5.875286876226152 8
STOP 21.08503125 21.89153125 0.8065 %done 6.362927639386172 9
STOP 22.637994791666